In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [38]:
df=pd.read_csv(r"C:\Users\DYNABOOK\Desktop\Hotel Reservations (1).csv")
# df.head()
df.shape

C:\Users\DYNABOOK\AppData\Local\Temp\ipykernel_17512\1836747864.py:1: DtypeWarning:

Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.



(36275, 21)

In [4]:
df.columns

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'date', 'arrival_year',
       'arrival_month', 'arrival_date', 'market_segment_type',
       'repeated_guest', 'no_of_previous_cancellations',
       'no_of_previous_bookings_not_canceled', 'avg_price_per_room',
       'no_of_special_requests', 'booking_status', 'no_of_people'],
      dtype='object')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36239 entries, 0 to 36274
Data columns (total 21 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36238 non-null  object 
 1   no_of_adults                          36238 non-null  float64
 2   no_of_children                        36238 non-null  float64
 3   no_of_weekend_nights                  36238 non-null  float64
 4   no_of_week_nights                     36238 non-null  float64
 5   type_of_meal_plan                     36238 non-null  object 
 6   required_car_parking_space            36238 non-null  float64
 7   room_type_reserved                    36238 non-null  object 
 8   lead_time                             36238 non-null  float64
 9   date                                  2626 non-null   object 
 10  arrival_year                          36238 non-null  float64
 11  arrival_month       

In [5]:
numerical_cols = df.select_dtypes(include=["number"]).columns
numerical_cols.shape
# numerical_cols

(15,)

## Let us check for the Duplicate values

In [49]:
# df.type_of_meal_plan.unique()
# df.isnull().sum()

In [43]:
df.duplicated().sum()

0

## Let us find the type of room,meals they provide

In [37]:
df.groupby("room_type_reserved")[["Booking_ID"]].count()
room_df=df.groupby("room_type_reserved")[["Booking_ID"]].count().reset_index()
# room_df.head()

In [9]:
fig = px.bar(room_df, x='room_type_reserved', y='Booking_ID',title="Room Types")
fig.show()

In [ ]:
# market_df=df.groupby("market_segment_type").count()
market_df = df.groupby(["market_segment_type", "room_type_reserved"]).size().reset_index(name="count")

fig = px.bar(market_df, 
             x="market_segment_type", 
             y="count",
             color="room_type_reserved", 
             barmode="group",
             title="Bookings by Market Segment and Room Type",
             height=500)
fig.show()

In [36]:
df.groupby("type_of_meal_plan")[["Booking_ID"]].count()
meal_df=df.groupby("type_of_meal_plan")[["Booking_ID"]].count().reset_index()
# meal_df.head()


In [12]:
fig = px.bar(meal_df, x='type_of_meal_plan', y='Booking_ID',title="Meal types")
fig.show()

In [30]:
people_room_df=df.groupby(["room_type_reserved","no_of_people"]).count()
# people_room_df

In [14]:
df.arrival_year.unique()

array([2017., 2018.,   nan])

In [15]:
df.groupby("arrival_year")[["Booking_ID"]].count()

,Booking_ID
arrival_year,
2017.0,6514
2018.0,29724


In [16]:
df.groupby("arrival_year")[["avg_price_per_room"]].sum()

,avg_price_per_room
arrival_year,
2017.0,586441.73
2018.0,3161917.67


In [56]:
df.loc[df["booking_status"]=="Not_Canceled"].groupby(["arrival_year","arrival_month"]).agg(
    total_sales_per_year=("avg_price_per_room", "sum"),
    total_bookings=("Booking_ID", "count"),
)
price_df=df.loc[df["booking_status"]=="Not_Canceled"].groupby(["arrival_year","arrival_month"]).agg(total_sales_per_year=("avg_price_per_room", "sum"),total_bookings=("Booking_ID", "count")).reset_index()
# price_df
price_df.arrival_year=price_df.arrival_year.astype(str)

In [57]:
fig = px.bar(price_df, x='arrival_year', y='total_sales_per_year',title="Arrival Year vs Total Sales",color="arrival_month")
fig.show()

In [58]:
fig = px.scatter(price_df, x="arrival_month", y="total_sales_per_year", color="arrival_year",
                 size='total_bookings',title="Sales over month")
fig.show()

## Here we can infer that this dataset has only 1.5 years of hotel reservation data 2017(7-12) and 2018,and 2018 having the highest sales.

In [35]:
lead_df=df.groupby(["lead_time","market_segment_type"])[["lead_time","market_segment_type"]].count()
# lead_df.shape
lead_df

lead2_df=pd.pivot_table(df,columns="market_segment_type",index="lead_time",values="Booking_ID",aggfunc="count")
lead2_df.fillna(0)

lead=df.groupby(["lead_time","market_segment_type"]).count()
# lead.head()

In [53]:
fig = px.histogram(lead_df, x="lead_time", nbins=40)
fig.show()

In [ ]:
booking_market_df=df.groupby(["booking_status","market_segment_type"])[["Booking_ID"]].count()
# booking_market_df


In [33]:
lead_time_count=df.groupby(["market_segment_type","lead_time"])["Booking_ID"].count().unstack()
# lead_time_count

In [ ]:
lead_time_long = lead_time_count.stack().reset_index()
lead_time_long.columns = ["market_segment_type", "lead_time", "booking_count"]

fig = px.box(
    lead_time_long, 
    x="market_segment_type", 
    y="lead_time", 
    title="Distribution of Lead Time by Market Segment Type",
    labels={"lead_time": "Lead Time (Days)", "market_segment_type": "Market Segment Type"}
)

fig.show()

### From this graph we can infer that Aviation has the lowest Lead Time, and Online has the highest lead time

In [25]:
# Merge with the original dataset to include 'booking_status'
lead_time_long = lead_time_long.merge(df[['market_segment_type', 'lead_time', 'booking_status']], 
                                      on=['market_segment_type', 'lead_time'], 
                                      how='left')

In [50]:
fig = px.box(
    lead_time_long, 
    x="market_segment_type", 
    y="lead_time", 
    color="booking_status",  
    title="Lead Time by Market Segment Type & Booking Status",
    labels={"lead_time": "Lead Time (Days)", "market_segment_type": "Market Segment Type"},
    category_orders={"booking_status": ["Not Canceled", "Canceled"]}  ,
    
)

fig.show()

### From the above graph we can see that the bookings made earlier have lesser probability to get canceled.

In [27]:
fig = px.box(df, x="room_type_reserved", y="lead_time", color="room_type_reserved",
             title="Lead Time Distribution by Room Type",
             labels={"lead_time": "Lead Time (Days)", "room_type_reserved": "Room Type"})
fig.show()

In [52]:
cancel_rate = df.groupby(["room_type_reserved", "booking_status"])["Booking_ID"].count().reset_index()

fig = px.bar(cancel_rate, x="room_type_reserved", y="Booking_ID", color="booking_status",
             title="Cancellation Rate by Room Type",
             labels={"booking_id": "Number of Bookings", "room_type_reserved": "Room Type", "booking_status": "Booking Status"},
             barmode="stack")
fig.show()